In [1]:
!wget https://raw.githubusercontent.com/anjelammcgraw/LLM-Zoomcamp/main/01%20intro/minisearch.py

--2024-06-25 05:57:07--  https://raw.githubusercontent.com/anjelammcgraw/LLM-Zoomcamp/main/01%20intro/minisearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minisearch.py.1’

minisearch.py.1     100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-25 05:57:07 (24.4 MB/s) - ‘minisearch.py.1’ saved [3832/3832]



In [2]:
import minisearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
index = minisearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [35]:
q = 'the course has already started, can I still enroll?'

In [36]:
index.fit(documents)

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"The ability to enroll in a course after it has already started depends on several factors, including the policies of the institution offering the course and the nature of the course itself. Here are some steps you can take to find out if late enrollment is possible:\n\n1. **Check the Course's Enrollment Policies:**\n   - Visit the institution’s website and look for information regarding late enrollment or add/drop deadlines. Some institutions have specific deadlines after which no additional enrollments are allowed.\n\n2. **Contact the Course Instructor:**\n   - Reach out to the instructor of the course directly. They might be flexible and allow you to join even if the course has started, especially if you have a compelling reason.\n\n3. **Speak with Academic Advisors:**\n   - Academic advisors can provide detailed information about your options and any potential consequences of late enrollment.\n\n4. **Consider the Course Format:**\n   - Some courses, especially those that are online

In [15]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

"To run Kafka, follow these steps depending on whether you are using Java or Python:\n\n### For Java:\nGo to your project directory and execute the following command in the terminal:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\nMake sure to replace `<jar_name>` with the actual name of your JAR file.\n\n### For Python:\n1. **Create a virtual environment** (if you haven't already):\n   ```\n   python -m venv env\n   source env/bin/activate  # For MacOS/Linux\n   env\\Scripts\\activate  # For Windows\n   pip install -r ../requirements.txt\n   ```\n\n2. **Activate the virtual environment** every time you need to run your Python file:\n   ```\n   source env/bin/activate  # For MacOS/Linux\n   env\\Scripts\\activate  # For Windows\n   ```\n\n3. **Deactivate the virtual environment** when you are done:\n   ```\n   deactivate\n   ```\n\nEnsure that you have all required dependencies installed and Docker images up and running as nec

In [20]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course after it has started. You will be eligible to submit the homework, but you need to be mindful of the deadlines for turning in the final projects.'

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [40]:
from elasticsearch import Elasticsearch

In [41]:
es_client = Elasticsearch('http://localhost:9200') 

In [42]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [43]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [44]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.06it/s]


In [46]:
query = 'I just disovered the course. Can I still join it?'

In [47]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [48]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
rag(query)

"Yes, you can still join the course even if you discovered it after it started. You're eligible to submit the homeworks regardless of registration. However, be mindful of the deadlines for submitting the final projects to avoid leaving everything to the last minute."